In [1]:
import os
import re
from collections import namedtuple
from datetime import date, datetime, time, timezone
from math import inf
from pathlib import Path
from tempfile import TemporaryDirectory
from xml.etree import ElementTree
from zipfile import ZipFile

from swarmclient.client import SwarmClient, SwarmFile
from swarmclient.config import ClientConfig

In [2]:
datadir = Path('~/Workspace/data').expanduser()

In [3]:
testfile = Path(datadir, 'SW_OPER_MPLAORBPRE_20191101T000000_20191206T000000_2172.ZIP')

In [4]:
def get_orbit_data(satellite, batch_size=50, max_files=None):
    
    if satellite not in ('A', 'B', 'C'):
        raise ValueError(
            'Satellite ID must be A, B or C, not %s' % satellite
        )
    if max_files is None:
        max_files = inf

    cc = ClientConfig()
    cln = SwarmClient(cc.default_site, *cc.get_site(cc.default_site).values())

    path = f'Auxiliary/Sat_{satellite}/MPL{satellite}ORBPRE'

    files = [elm for n, elm in enumerate(cln.search(path, file_type=f'MPL{satellite}ORBPRE')) if n < max_files]
    
    with TemporaryDirectory() as tmp:
        print('downloading files to %s' % tmp)
        while len(files):
            cln.download(files[:batch_size], tmp)
            files = files[batch_size:]
        print('done')
            
    

In [3]:
try:
    get_orbit_data('A', max_files=50)
except Exception as err:
    print(err)

downloading files to /var/folders/kr/0v3bs2p53wdd98x4103pjmdm0000gn/T/tmpkbdqn1by


SW_OPER_MPLAORBPRE_20131122T131347_20131207T131347_0014.ZIP: 100%|██████████| 17.8k/17.8k [00:00<00:00, 79.8kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131202T133247_0016.ZIP: 100%|██████████| 12.4k/12.4k [00:00<00:00, 66.5kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131202T232900_0017.ZIP: 100%|██████████| 12.9k/12.9k [00:00<00:00, 70.8kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131203T002900_0018.ZIP: 100%|██████████| 13.0k/13.0k [00:00<00:00, 69.8kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131203T002900_0019.ZIP: 100%|██████████| 13.0k/13.0k [00:00<00:00, 70.1kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131203T055900_0020.ZIP: 100%|██████████| 13.2k/13.2k [00:00<00:00, 70.0kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20131203T055900_0021.ZIP: 100%|██████████| 13.0k/13.0k [00:00<00:00, 71.7kB/s]
SW_OPER_MPLAORBPRE_20131122T131447_20140102T235900_0021.ZIP: 100%|██████████| 47.2k/47.2k [00:00<00:00, 173kB/s] 
SW_OPER_MPLAORBPRE_20131122T131447_20140102T235900_0022.ZIP: 100%|██████████| 47.2k/47.2

done


In [7]:
class OrbitStateVector:
    
    def __init__(self, tai, utc, ut1, absolute_orbit, x, y, z, vx, vy, vz, quality):
        self.__tai = self.__to_datetime(tai)
        self.__utc = self.__to_datetime(utc)
        self.__ut1 = self.__to_datetime(ut1)
        self.__absolute_orbit = int(absolute_orbit)
        self.__x = float(x)
        self.__y = float(y)
        self.__z = float(z)
        self.__vx = float(vx)
        self.__vy = float(vy)
        self.__vz = float(vz)
        self.__quality = int(quality)
    
    def __to_datetime(self, value):
        if isinstance(value, date):
            if not hasattr(value, 'hour'):
                value = datetime.combine(value, time())
        elif isinstance(value, str):
            value = datetime.strptime(value[4:], '%Y-%m-%dT%H:%M:%S.%f')
        return value.replace(tzinfo=timezone.utc)
            
    
    def __repr__(self):
        return f'<Orbit({self.__absolute_orbit})>'
    
    @property
    def times(self):
        return {
            'TAI': self.__tai,
            'UT1': self.__ut1,
            'UTC': self.__utc
        }

    @property
    def position(self):
        return {
            'x': self.__x,
            'y': self.__y,
            'z': self.__z
        }

    @property
    def velocity(self):
        return {
            'vx': self.__vx,
            'vy': self.__vy,
            'vz': self.__vz
        }

    @property
    def absolute_orbit(self):
        return self.__absolute_orbit
    
    @property
    def quality(self):
        return self.__quality
    
    

In [8]:
osv = OrbitStateVector(
    tai='TAI=2019-11-01T00:08:00.538843',
    utc='UTC=2019-11-01T00:07:23.538843',
    ut1='UT1=2019-11-01T00:07:23.378921',
    absolute_orbit='+33383',
    x='-6752454.829',
    y='+0925206.375',
    z='-0010660.291',
    vx='+0018.385635',
    vy='+0145.048283',
    vz='+7641.120229',
    quality='0000000000000'
)

In [9]:
osv

<Orbit(33383)>

In [10]:
osv.times

{'TAI': datetime.datetime(2019, 11, 1, 0, 8, 0, 538843, tzinfo=datetime.timezone.utc),
 'UT1': datetime.datetime(2019, 11, 1, 0, 7, 23, 378921, tzinfo=datetime.timezone.utc),
 'UTC': datetime.datetime(2019, 11, 1, 0, 7, 23, 538843, tzinfo=datetime.timezone.utc)}

In [11]:
osv.absolute_orbit

33383

In [12]:
osv.position

{'x': -6752454.829, 'y': 925206.375, 'z': -10660.291}

In [13]:
osv.velocity

{'vx': 18.385635, 'vy': 145.048283, 'vz': 7641.120229}

In [51]:
class PredictedOrbitFile:
    
    __pattern = re.compile(
        r'SW_(?P<file_class>\w{4})_(?P<file_type>MPL(?P<sat_id>[ABC])ORBPRE)_'
        r'(?P<validity_start>\d{8}T\d{6})_(?P<validity_stop>\d{8}T\d{6})_'
        r'(?P<version>\d{4})\.(?P<suffix>ZIP|EEF)'
    )
    
    def __init__(self, filename):
        p = Path(filename)
        m = self.__pattern.match(p.name)
        if not m:
            raise ValueError('invalid file name: %s' % p.name)
        self.__properties = m.groupdict()
        self.__parent = p.parent
        self.__name = p.name

    @property
    def mission(self):
        return 'SW'
    
    @property
    def file_class(self):
        return self.__properties['file_class']
    
    @property
    def file_type(self):
        return self.__properties['file_type']
    
    @property
    def sat_id(self):
        return self.__properties['sat_id']

    @property
    def suffix(self):
        return self.__properties['suffix']
    


In [52]:
p = PredictedOrbitFile(testfile.name)

In [53]:
p.file_class

'OPER'

In [54]:
p.file_type

'MPLAORBPRE'

In [55]:
p.sat_id

'A'

In [56]:
p.suffix

'ZIP'